# Atwater Subsistence Diet Optimization

Code by Joyce Li

In [2]:
!pip install -r requirements.txt
#!pip install --upgrade pip
import pandas as pd
import numpy as np

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Pulling FDC Data

In [3]:
user = "joyceml"

In [4]:
apikey = {'joyceml':"r4qly7ALQ3bhuG1rdr63aCqUZYieS69MEwaXDogb"}

### Whenever adding a new ingredient: need to 
1. pull from FDC data
2. put df name into lista, put Sheets column name into food_list
3. Fill in Google Sheets

In [45]:
#Identify Best-Fitting FDCID (choose based on description)
import fooddatacentral as fdc

df1 = fdc.search(apikey[user],"dried shitake mushrooms")
df1.head()

,additionalDescriptions,allHighlightFields,commonNames,dataType,description,fdcId,foodCode,ndbNumber,publishedDate,scientificName,score
0,NaN,,NaN,SR Legacy,"Mushrooms, shiitake, dried",168436,NaN,11268,2019-04-01,Lentinus edodes,618.43524
1,shiitake,,NaN,Survey (FNDDS),"Mushroom, Asian, cooked, from dried",343137,75219100,NaN,2019-04-01,NaN,513.16090
2,commercially canned lowfat gravy,,NaN,Survey (FNDDS),"Gravy, mushroom",338992,28500100,NaN,2019-04-01,NaN,354.18362
3,NaN,,NaN,Survey (FNDDS),"Mushrooms, pickled",343664,75505000,NaN,2019-04-01,NaN,354.18362
4,NaN,,NaN,Survey (FNDDS),"Mushrooms, raw",342623,75115000,NaN,2019-04-01,NaN,354.18362


In [46]:
#Pull Specific Vegetables from FDC Dataset

collard_greens = fdc.nutrients(apikey[user],fdc_id=380297).rename(columns={"Quantity": "Collard Greens", "Units": "collard_greens_u"}) #walmart
carrots = fdc.nutrients(apikey[user],fdc_id=658616).rename(columns={"Quantity": "Carrots", "Units": "carrot_u"})
cabbage = fdc.nutrients(apikey[user],fdc_id=169975).rename(columns={"Quantity": "Cabbage", "Units": "cabbage_u"})
lettuce = fdc.nutrients(apikey[user],fdc_id=342618).rename(columns={"Quantity": "Lettuce", "Units": "lettuce_u"})
cauliflower = fdc.nutrients(apikey[user],fdc_id=528384).rename(columns={"Quantity": "Cauliflower", "Units": "cauliflower_u"})
diced_tomatoes = fdc.nutrients(apikey[user],fdc_id=371622).rename(columns={"Quantity": "Diced Tomatoes", "Units": "tomatoes_u"})
rice = fdc.nutrients(apikey[user],fdc_id=450892).rename(columns={"Quantity": "Rice", "Units": "rice_u"})
#nuts = fdc.nutrients(apikey[user],fdc_id=570267).rename(columns={"Quantity": "nuts_q", "Units": "nuts_u"})
bananas = fdc.nutrients(apikey[user],fdc_id=173944).rename(columns={"Quantity": "Bananas", "Units": "bananas_u"})
tofu = fdc.nutrients(apikey[user],fdc_id=496446).rename(columns={"Quantity": "Tofu", "Units": "tofu_u"})
eggs = fdc.nutrients(apikey[user],fdc_id=442160).rename(columns={"Quantity": "Eggs", "Units": "eggs_u"})
cereal = fdc.nutrients(apikey[user],fdc_id=653426).rename(columns={"Quantity": "Cereal", "Units": "cereal_u"})
mushrooms = fdc.nutrients(apikey[user],fdc_id=168436).rename(columns={"Quantity": "Dried Shitake", "Units": "mushrooms_u"})
rice_wrap = fdc.nutrients(apikey[user],fdc_id=340211).rename(columns={"Quantity": "Rice Wrap", "Units": "rice_wrap_u"})
noodles = fdc.nutrients(apikey[user],fdc_id=373372).rename(columns={"Quantity": "Noodles", "Units": "noodles_u"})
peanut_butter = fdc.nutrients(apikey[user],fdc_id=357668).rename(columns={"Quantity": "Peanut Butter", "Units": "peanut_butter_u"})
cilantro = fdc.nutrients(apikey[user],fdc_id=530557).rename(columns={"Quantity": "Cilantro", "Units": "cilantro_u"})
cucumber = fdc.nutrients(apikey[user],fdc_id=342612).rename(columns={"Quantity": "Cucumbers", "Units": "cucumber_u"})

veggie_list = [collard_greens, carrots, cabbage, lettuce, cauliflower, diced_tomatoes, rice, tofu, bananas, eggs, cereal, mushrooms, rice_wrap, noodles, peanut_butter, cilantro, cucumber]


In [48]:
#Matrix Manipulation

#Combine lists that are pulled from FDC Dataset
start = collard_greens.reset_index()
for i in veggie_list[1:]:
    veg_t = start.merge(i.reset_index(), how = 'outer', on = 'index')
    start = veg_t
    

#make sure units are the same (convert kJ to kcal)
#one_kj = 0.239006 kcal
kj = veg_t['Cabbage'][3]
veg_t['Cabbage'][3] = kj*0.239006
kj = veg_t['Bananas'][3]
veg_t['Bananas'][3] = kj*0.239006
table = veg_t.set_index('index')

unit_list = list(veg_t['cabbage_u'])
unit_list[3] = "kcal"

#get rid of units columns
keep = list(np.arange(0, len(veggie_list)*2,2))

total_table = table.iloc[:, keep]
total_table = total_table.fillna(0)
total_table['meal_total'] = total_table.sum(axis=1)
total_table['units'] = unit_list

#table with rows as type of nutrient, columns are each vegetable and a total
total_table

#export = total_table.reset_index()
D =total_table.drop(columns={'meal_total', 'units'})
D

    

#carrots.reset_index().merge(collard_greens.reset_index(), how= 'outer', on = 'index')

/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/srv/app/venv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Collard Greens,Carrots,Cabbage,Lettuce,Cauliflower,Diced Tomatoes,Rice,Tofu,Bananas,Eggs,Cereal,Dried Shitake,Rice Wrap,Noodles,Peanut Butter,Cilantro,Cucumbers
index,,,,,,,,,,,,,,,,,
"Calcium, Ca",78.00,38.00,40.000000,18.000,0.00,16.00,44.00,118.00,5.000000,40.00,0.000,11.000,10.000,0.00,0.00,0.0,14.000
"Carbohydrate, by difference",3.12,11.54,5.800000,2.970,4.71,7.14,80.00,3.53,22.840000,0.00,89.660,75.370,72.540,77.78,21.88,0.0,2.160
Cholesterol,0.00,0.00,0.000000,0.000,0.00,0.00,0.00,0.00,0.000000,370.00,0.000,0.000,0.000,0.00,0.00,0.0,0.000
Energy,16.00,38.00,24.617618,14.000,29.00,32.00,356.00,82.00,88.671226,140.00,379.000,1238.000,329.000,370.00,625.00,0.0,12.000
"Fatty acids, total monounsaturated",0.00,0.00,0.017000,0.006,0.00,0.00,0.00,0.00,0.032000,0.00,0.000,0.317,0.376,0.00,0.00,0.0,0.002
"Fatty acids, total polyunsaturated",0.00,0.00,0.017000,0.074,0.00,0.00,0.00,0.00,0.073000,0.00,0.000,0.151,0.322,0.00,0.00,0.0,0.003
"Fatty acids, total saturated",0.00,0.00,0.034000,0.018,0.00,0.00,0.00,0.47,0.112000,3.00,0.000,0.225,0.328,0.00,10.94,0.0,0.013
"Fatty acids, total trans",0.00,0.00,0.000000,0.000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000,0.000,0.000,0.00,0.00,0.0,0.000
"Fiber, total dietary",1.60,3.80,2.500000,1.200,1.20,1.60,2.20,1.20,2.600000,0.00,3.400,11.500,2.100,3.70,6.20,0.0,0.700


In [49]:
#export.to_csv('total_table.csv', index = False)

# Prices & Google Sheets

In [50]:
SHEETs = [# Stigler's foods, modern prices
          ("1-SnIIK7ZxtXVvCJUYK9ory2fFhRCICWOQ1NJVNkPb4k","Sheet1"), 
         ]

In [51]:
#Food Price Data
import warnings

DFs = []
#for csv in CSVs:  # Uncomment to use a list of csv files as inputs
#  DFs.append(pd.read_csv(csv,dtype=str))

try:
    if len(SHEETs):
        for ID, RANGE_NAME in SHEETs:
            try:
                if "docs.google.com" in ID:
                    sheet = "%s&output=csv" % ID
                else:
                    sheet = "https://docs.google.com/spreadsheet/ccc?key=%s&output=csv" % ID
                DFs.append(pd.read_csv(sheet))
            except ParserError:
                warnings.warn("Can't read sheet at https://docs.google.com/spreadsheets/d/%s.\nCheck Sharing settings, so that anyone with link can view?" % ID)
except NameError: # SHEETS not defined?
    pass

df = pd.concat(DFs,ignore_index=True,sort=False)
# Some columns which ought to be numeric are actually str; convert them
df['Price'] = df['Price'].astype(float)
df['Quantity'] = df['Quantity'].astype(float)
df['FDC'] = df['FDC'].astype(int)

df
pricing = df
pricing

,Food,Quantity,Units,Price,Date,Location,FDC
0,Collard Greens,1.0,lbs,0.860,3/2,Berkeley Bowl,380297
1,Carrots,1.0,lbs,0.920,3/2,Berkeley Bowl,658616
2,Cabbage,1.0,lbs,0.990,3/2,Berkeley Bowl,169975
3,Lettuce,1.0,lbs,0.350,3/2,Berkeley Bowl,342618
4,Cauliflower,1.0,lbs,1.330,3/2,Berkeley Bowl,528384
5,Diced Tomatoes,1.0,lbs,0.380,3/2,Berkeley Bowl,371622
6,Rice,1.0,lbs,1.760,3/2,Berkeley Bowl,450892
7,Tofu,1.0,lbs,2.900,3/2,Berkeley Bowl,570267
8,Bananas,1.0,lbs,0.590,3/2,Berkeley Bowl,173944
9,Eggs,1.0,lbs,1.128,3/2,Berkeley Bowl,442160


In [52]:
#Checking FDC Requirements
#The FDC database basically wants everything in either hundreds of grams (hectograms) or hundreds of milliliters (deciliters).
pricing['FDC Quantity'] = pricing[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
pricing['FDC Price'] = pricing['Price']/pricing['FDC Quantity']

pricing.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = pricing.groupby('Food',sort=False)['FDC Price'].min()

Prices

Food
Collard Greens    0.18959754547899468 / hectogram
Carrots           0.20282528121008736 / hectogram
Cabbage           0.21825763956302877 / hectogram
Lettuce           0.07716179176470714 / hectogram
Cauliflower        0.2932148087058872 / hectogram
Diced Tomatoes    0.08377565963025348 / hectogram
Rice               0.3880135814453845 / hectogram
Tofu               0.6393405603361448 / hectogram
Bananas           0.13007273468907776 / hectogram
Eggs              0.24868143174454185 / hectogram
Cereal             0.8818490487395102 / hectogram
Dried Shitake      3.3069339327731635 / hectogram
Rice Wrap          1.3095458373781728 / hectogram
Noodles            0.9546015952605198 / hectogram
Peanut Butter     0.49383546729412575 / hectogram
Cilantro           0.1719605645042045 / hectogram
Cucumbers         0.32848877065546755 / hectogram
Name: FDC Price, dtype: object

# Identify Constraints and Solve The Problem

In [53]:
# Choose sex/age group:
group = "F 19-30"

# Define *minimums*
bmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')[group]

# Define *maximums*
bmax = pd.read_csv('./diet_maximums.csv').set_index('Nutrition')[group]

# Inspect:
bmin

Nutrition
Energy                            2000.0
Protein                             46.0
Fiber, total dietary                28.0
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                            18.0
Magnesium, Mg                      310.0
Niacin                              14.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.1
Thiamin                              1.1
Vitamin A, RAE                     700.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      75.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)           90.0
Zinc, Zn                             8.0
Name: F 19-30, dtype: float64

In [54]:
c = Prices.apply(lambda x:x.magnitude).dropna()

# C is a list that is a compilation of what we have prices and nutritional info; shows list of ingredients and prices of amounts in Google Sheet

#use = list(set(c.index.tolist()).intersection(food_list))
#c = c[use]
c


Food
Collard Greens    0.189598
Carrots           0.202825
Cabbage           0.218258
Lettuce           0.077162
Cauliflower       0.293215
Diced Tomatoes    0.083776
Rice              0.388014
Tofu              0.639341
Bananas           0.130073
Eggs              0.248681
Cereal            0.881849
Dried Shitake     3.306934
Rice Wrap         1.309546
Noodles           0.954602
Peanut Butter     0.493835
Cilantro          0.171961
Cucumbers         0.328489
Name: FDC Price, dtype: float64

In [55]:
#D is nutritional info matrix
Aall = D

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Minimum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])

A

,Collard Greens,Carrots,Cabbage,Lettuce,Cauliflower,Diced Tomatoes,Rice,Tofu,Bananas,Eggs,Cereal,Dried Shitake,Rice Wrap,Noodles,Peanut Butter,Cilantro,Cucumbers
Nutrition,,,,,,,,,,,,,,,,,
Energy,16.00,38.00,24.617618,14.000,29.00,32.00,356.000,82.00,88.671226,140.00,379.000,1238.000,329.000,370.00,625.00,0.0,12.000
Protein,1.56,1.28,1.280000,0.900,1.18,0.79,6.670,10.59,1.090000,12.00,3.450,9.580,5.070,11.11,25.00,0.0,0.590
"Fiber, total dietary",1.60,3.80,2.500000,1.200,1.20,1.60,2.200,1.20,2.600000,0.00,3.400,11.500,2.100,3.70,6.20,0.0,0.700
"Folate, DFE",0.00,0.00,43.000000,29.000,0.00,0.00,0.000,0.00,20.000000,0.00,0.000,163.000,4.000,0.00,0.00,0.0,14.000
"Calcium, Ca",78.00,38.00,40.000000,18.000,0.00,16.00,44.000,118.00,5.000000,40.00,0.000,11.000,10.000,0.00,0.00,0.0,14.000
"Carbohydrate, by difference",3.12,11.54,5.800000,2.970,4.71,7.14,80.000,3.53,22.840000,0.00,89.660,75.370,72.540,77.78,21.88,0.0,2.160
"Iron, Fe",0.84,1.28,0.470000,0.410,0.00,0.29,4.000,3.39,0.260000,1.44,15.520,1.720,0.380,1.33,2.25,0.0,0.220
"Magnesium, Mg",0.00,0.00,12.000000,7.000,0.00,0.00,0.000,0.00,27.000000,0.00,0.000,132.000,30.000,0.00,0.00,0.0,12.000
Niacin,0.00,0.00,0.234000,0.123,0.00,0.00,4.444,0.00,0.665000,0.00,17.241,14.100,2.202,0.00,0.00,0.0,0.037


In [56]:
#Modifying Constraints to fit Stigler's constraints

b = pd.DataFrame(pd.concat([bmin,-bmax])) # Note sign change for max constraints
stiglers = ['Protein', 'Carbohydrate, by difference', 'Fiber, total dietary', 'Calcium, Ca', 'Iron, Fe', 'Vitamin A', 'Thiamin', 'Riboflavin', 'Niacin']
drop_list = ['Energy', 'Folate, DFE', 'Magnesium, Mg', 'Phosphorus, P', 'Potassium, K', 'Vitamin B-12', 'Vitamin B-6', 'Vitamin C, total ascorbic acid', 'Vitamin E (alpha-tocopherol)','Vitamin K (phylloquinone)', 'Zinc, Zn', 'Sodium, Na']

b = b.drop(drop_list)
A = A.drop(drop_list)


In [57]:
#SOLVE THE PROBLEM

from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(c, -A, -b, method='interior-point')

result

     con: array([], dtype=float64)
     fun: 4.008183342374733
 message: 'Optimization terminated successfully.'
     nit: 10
   slack: array([2.10365414e-09, 1.82409303e+01, 1.67713097e-08, 1.00158077e+02,
       7.90791975e+00, 1.07928955e-09, 1.23452712e-08, 3.20851305e-01,
       2.57454644e-07])
  status: 0
 success: True
       x: array([5.51019977e+00, 2.58446602e-10, 2.20705626e-10, 2.80000000e+01,
       7.30714981e-11, 8.84546611e-10, 1.31678008e+00, 8.35005062e-11,
       3.80570953e-10, 2.06652352e-01, 2.72851304e-01, 8.19567639e-12,
       1.82268138e-11, 2.73520901e-11, 1.52370333e-10, 0.00000000e+00,
       7.11832101e-11])

# Results

## Minimum Cost

In [58]:
print("Cost of diet for %s is $%4.2f per day." % (group,result.fun))

Cost of diet for F 19-30 is $4.01 per day.


## Suggested Diet

In [59]:
#DIET CONSISTS OF...
# Put back into nice series
diet = pd.DataFrame(result.x,index=c.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.


You'll be eating (in 100s of grams or milliliters):
                        0
Food                     
Collard Greens   5.510200
Carrots               NaN
Cabbage               NaN
Lettuce         28.000000
Cauliflower           NaN
Diced Tomatoes        NaN
Rice             1.316780
Tofu                  NaN
Bananas               NaN
Eggs             0.206652
Cereal           0.272851
Dried Shitake         NaN
Rice Wrap             NaN
Noodles               NaN
Peanut Butter         NaN
Cilantro              NaN
Cucumbers             NaN


## Total Nutrition Possible from ingredients


In [60]:
A.T.sum()

Nutrition
Protein                         92.140
Fiber, total dietary            45.500
Calcium, Ca                    432.000
Carbohydrate, by difference    481.040
Iron, Fe                        33.800
Niacin                          39.046
Riboflavin                       2.917
Thiamin                          1.582
Vitamin A, RAE                  37.000
dtype: float64

## Our Outcome vs. Recommended

In [61]:
A.shape, diet.shape
A.columns == diet.index
O =A.dot(diet)
E = np.abs(b)
tab = O.merge(E, on = 'Nutrition')
tab = tab.rename(columns = {0: 'Outcome', 'F 19-30': 'Recommended'})
tab

,Outcome,Recommended
Nutrition,,
Protein,46.000000,46.0
"Fiber, total dietary",46.240930,28.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",230.158077,130.0
"Iron, Fe",25.907920,18.0
Niacin,14.000000,14.0
Riboflavin,1.100000,1.1
Thiamin,1.420851,1.1
"Vitamin A, RAE",700.000000,700.0


## Constraining Nutrients

In [62]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())


Constraining nutrients are:
['Protein', 'Calcium, Ca', 'Niacin', 'Riboflavin', 'Vitamin A, RAE']
